# ml logic

## data.py

In [1]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer


In [2]:
def create_binary_df(file_path):

    '''
    Load the original dataframe,
    binary encode the 'Grapes' (part of features) and 'Harmonize' (becomes multi-label target y),
    remove all columns not needed for model training,
    return the binary encoded and cleaned dataframe
    '''

    #binary encoder for 'Harmonize' column (multi-label target)
    mlb_harm = MultiLabelBinarizer(sparse_output=False)
    #binary encoder for 'Grape' column (feature)
    mlb_grape = MultiLabelBinarizer(sparse_output=False)

    wine_df = pd.read_csv(file_path)

    #Drop addional columns not used for model
    wine_df = wine_df.drop(columns=['WineName', 'WineID','Code','Country','RegionID','RegionName','WineryID','Website','Vintages', 'WineryName'])

    # Binary encode grapes
    wine_df_bin = wine_df.join(pd.DataFrame(
        mlb_grape.fit_transform(eval(element) for element in wine_df.Grapes),
        index=wine_df.index,
        columns=mlb_grape.classes_
        ))
    wine_df_bin.drop(columns=['Grapes'], inplace=True)

    # Create a list of the kind of grapes that are mentioned less then 2.000 times
    grapes_list = wine_df_bin.iloc[:,16:].sum() # sum the number of times a grape is mentioned via column
    final_column_grapes = wine_df_bin.shape[1]

    # Binary encode Harmonize(kinds of food)
    wine_df_bin = wine_df_bin.join(pd.DataFrame(
        mlb_harm.fit_transform(eval(element) for element in wine_df.Harmonize),
        index=wine_df.index,
        columns=mlb_harm.classes_
        ))
    wine_df_bin.drop(columns=['Harmonize'], inplace=True)

    # Create a list of the kind of grapes that are mentioned less then 2.000 times
    harm_list = wine_df_bin.iloc[:,(final_column_grapes+1):].sum() # sum the number of times a food is mentioned via column
    harm_to_drop = harm_list[harm_list<=15_000].index.to_list() # create a list withe kind of food mentioned less then 50 times
    wine_df_bin.drop(columns=harm_to_drop, inplace=True) # drop columns with food not mentioned more then 50 times
    wine_df_bin = wine_df_bin[wine_df_bin.iloc[:,(final_column_grapes+1):].eq(1).any(axis=1)] # drop wines which are not represented by a food anymore

    return wine_df_bin, final_column_grapes

## model.py

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np

from skmultilearn.problem_transform import LabelPowerset

import pickle

In [4]:
def create_X_train_y_train(df, grape_column, test_size=0.3):

    '''
    Read in transformed data frame and the final column for grapes (last feature column of data frame)
    Return X_train, X_test, y_train, y_test
    '''

    X = df.iloc[:,:grape_column-1]
    X = X.drop(columns = 'Elaborate')
    y = df.iloc[:,grape_column:]
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=test_size)
    return X_train, X_test, y_train, y_test

In [5]:
def train_model(X_train, y_train):

    '''
    Create pipeline for feature transformation and
    model training
    '''

    # Create binary classifier
    # Define which columns need to be encoded
    cat_cols = make_column_selector(dtype_include='object')
    num_cols = make_column_selector(dtype_include='number')
    cat_pre = make_pipeline(OneHotEncoder(sparse_output=False, handle_unknown='ignore'),
                            MinMaxScaler())
    cat_num = MinMaxScaler()

    # Create preprocessor pipeline
    preprocessing = make_column_transformer((cat_pre, cat_cols),(cat_num, num_cols))

    classifier = LabelPowerset(RandomForestClassifier(max_features=1, min_samples_split=10,
                                                      n_jobs=-1, random_state=42))

    pipeline = make_pipeline(preprocessing, classifier)
    return pipeline.fit(X_train, y_train)


In [6]:
def save_model(model, model_save_path):
    # Save the model as a pickle file
    with open(model_save_path, 'wb') as f:
        pickle.dump(model, f)

In [7]:
def open_model(model_open_path):
    # Open the model as a pickle file
    with open(model_open_path, "rb") as f:
        model = pickle.load(f)
    return model

In [23]:
def create_X_pred(Type: str,
            ABV: float,
            Body: str,
            Acidity: str,
            grapes: list):

    '''
    Return a X_pred readable by the model by populating all grape columns not
    listed with 0 and the ones listed with 0
    '''

    X_pred = pd.DataFrame.from_dict({'Type': [Type],
                                        'Body': [Body],
                                        'Acidity': [Acidity],
                                        'ABV': [ABV]},
                                    orient='columns')

    # list of all grape columns (binary encoded)
    grapes_columns = ['"BiancodAlessano"', '"LAcadieBlanc"', '"LendelEl"', '"LoindelOeil"', '"NerodAvola"', '"PineauDAunis"', '"RoussetteDAyze"', '"TrebbianodAbruzzo"', 'Abbuoto', 'Abouriou', 'Abrostine', 'Acolon', 'Agiorgitiko', 'Aglianico', 'Aidani', 'Airen', 'Albalonga', 'Albana', 'Albanella', 'Albariño', 'Albarola', 'Albarossa', 'AlbarínBlanco', 'Albillo', 'AlbilloCrimean', 'AlbilloMayor', 'AlbillodeAlbacete', 'Aleatico', 'AlfrocheiroPreto', 'Alibernet', 'AlicanteBouschet', 'AlicanteGanzin', 'Aligoté', 'Altesse', 'Alvarelhão', 'Alvarinho', 'Amigne', 'Ancellotta', 'Ansonica', 'AntãoVaz', 'Aragonez', 'Aramon', 'Arbane', 'Areni', 'Argaman', 'Arinarnoa', 'Arinto', 'ArintodeBucelas', 'ArintodosAçores', 'Arneis', 'Arnsburger', 'Arriloba', 'AspiranBouschet', 'AsprinioBianco', 'AssarioBranco', 'Assyrtiko', 'Athiri', 'Aurore', 'Avanà', 'Avesso', 'Avgoustiatis', 'AzalBranco', 'AzalTinto', 'Babić', 'Bacchus', 'BacoNoir', 'Baga', 'Barbarossa', 'Barbera', 'Barcelo', 'Barsaglina', 'BastardoMagarachsky', 'Batoca', 'Bellone', 'Bianca', 'Biancame', 'BianchettaTrevigiana', 'Biancolella', 'Bical', 'BlackQueen', 'Blauburger', 'Blauburgunder', 'BlauerPortugieser', 'Blaufränkisch', 'BoalBranco', 'Bobal', 'Bogazkere', 'BombinoBianco', 'BombinoNero', 'Bonamico', 'Bonarda', 'Bordô', 'Borraçal', 'Bosco', 'Bourboulenc', 'Bovale', 'Brachetto', 'Braquet', 'Braucol', 'Brianna', 'Bronner', 'BrunArgenté', 'Bruñal', 'Bual', 'BudaiZöld', 'Bukettraube', 'BurgundMare', 'BusuioacadeBohotin', 'BăbeascăNeagră', 'CabernetBlanc', 'CabernetCortis', 'CabernetCubin', 'CabernetDorsa', 'CabernetFranc', 'CabernetJura', 'CabernetMitos', 'CabernetRuby', 'CabernetSauvignon', 'CabernetSeverny', 'Cagnulari', 'CaiñoBlanco', 'CaiñoTinto', 'CalabresediMontenuovo', 'Caladoc', 'Calkarasi', 'Callet', 'Camarate', 'CanaioloBlanco', 'CanaioloNero', 'Cannonau', 'Carignan/Cariñena', 'Carmenère', 'Carricante', 'Casavecchia', 'Cascade', 'Casetta', 'Castelão', 'CatarrattoBianco', 'Catawba', 'CayugaWhite', 'Cencibel', 'Centesiminio', 'CercealBranco', 'Cesanese', 'Chambourcin', 'Chancellor', 'Charbono', 'Chardonel', 'Chardonnay', 'ChardonnayMusqué', 'Chasan', 'Chasselas', 'Chatus', 'Chenanson', 'CheninBlanc', 'Chinuri', 'Cienna', 'Ciliegiolo', 'Cinsault', 'Clairette', 'Cococciola', 'CodadiVolpeBianca', 'Colobel', 'Colombard', 'Coloraillo', 'ColorinodelValdarno', 'Concord', 'CorintoNero', 'Cornalin', 'Cornifesto', 'CorotNoir', 'Cortese', 'Corvina', 'Corvinone', 'Couderc', 'Counoise', 'CriollaGrande', 'Croatina', 'Crouchen', 'Cynthiana', 'CôdegadeLarinho', 'Côt', 'Dafni', 'Dakapo', 'DeChaunac', 'Debina', 'Diagalves', 'Dimiat', 'Dimrit', 'Dindarella', 'Diolinoir', 'Dolcetto', 'Domina', 'DonaBlanca', 'DonzelinhoBranco', 'DonzelinhoTinto', 'Dornfelder', 'Drupeggio', 'Dunkelfelder', 'Duras', 'Durella', 'Durif', 'DzvelshaviObchuri', 'Edelweiss', 'Egiodola', 'Ehrenfelser', 'EmeraldRiesling', 'Emir', 'Enantio', 'Encruzado', 'Erbaluce', 'Espadeiro', 'Falanghina', 'FalanghinaBeneventana', 'Famoso', 'Favorita', 'Fenile', 'FerServadou', 'FernãoPires', 'FeteascaAlba', 'FeteascaNeagra', 'FeteascaRegala', 'Fiano', 'Flora', 'FogliaTonda', 'Fokiano', 'Folgasao', 'FolleBlanche', 'FonteCal', 'Fragolino', 'Francusa', 'Frappato', 'Fredonia', 'Freisa', 'Friulano/Sauvignonasse', 'Frontenac', 'FruhroterVeltliner', 'Frühburgunder', 'Fumin', 'FuméBlanc', 'Furmint', 'Gaglioppo', 'Gaidouria', 'Galotta', 'Gamaret', 'GamayNoir', 'GamayTeinturierdeBouze', 'GambadiPernice', 'Garanoir', 'Garganega', 'Garnacha', 'GarnachaBlanca', 'GarnachaPeluda', 'GarnachaRoja', 'GarnachaTinta', 'GarnachaTintorera', 'GarridoFino', 'GelberMuskateller', 'Gewürztraminer', 'Gigiac', 'Ginestra', 'Girgentina', 'GiròBlanc', 'Glera/Prosecco', 'Godello', 'GoldTraminer', 'Goldburger', 'Golubok', 'Gorgollasa', 'GoruliMtsvane', 'Gouveio', 'GouveioReal', 'Graciano', 'GrandNoir', 'GrasadeCotnari', 'Grauburgunder', 'Grecanico', 'Grechetto', 'GrechettoRosso', 'Greco', 'GrecoBianco', 'GrecoNero', 'Grenache', 'GrenacheBlanc', 'GrenacheGris', 'Grignolino', 'Grillo', 'Gringet', 'Grolleau', 'Groppello', 'GrosManseng', 'GrosVerdot', 'GrünerVeltliner', 'Guardavalle', 'Gutedel', 'Hanepoot', 'Helios', 'Hibernal', 'HondarrabiBeltza', 'HondarrabiZuri', 'HumagneBlanche', 'HumagneRouge', 'Huxelrebe', 'Hárslevelű', 'IncrocioManzoni', 'Inzolia', 'IrsaiOliver', 'Isabella', 'Jacquère', 'Jaen', 'Jampal', 'Johannisberg', 'Johanniter', 'JuanGarcia', 'Kabar', 'Kadarka', 'Kakhet', 'Kakotrygis', 'KalecikKarasi', 'Kangun', 'Karasakiz', 'Karmahyut', 'Katsano', 'Keratsuda', 'Kerner', 'Khikhvi', 'Királyleányka', 'Kisi', 'Klevner', 'KokurBely', 'Koshu', 'Kotsifali', 'KrasnostopAnapsky', 'KrasnostopZolotovsky', 'Kratosija', 'Krstac', 'Kydonitsa', 'Kékfrankos', 'Lacrima', 'Lafnetscha', 'Lagrein', 'Lambrusco', 'Lampia', 'LandotNoir', 'Lauzet', 'Leanyka', 'Lefkada', 'Lemberger', 'Lenoir', 'LeonMillot', 'Liatiko', 'Limnio', 'Limniona', 'ListanNegro', 'Lorena', 'Loureiro', 'Macabeo', 'MadeleineAngevine', 'MaglioccoCanino', 'Malagouzia', 'Malbec', 'MalboGentile', 'Malvar', 'Malvasia', 'MalvasiaBiancaLunga', 'MalvasiaFina', 'MalvasiaIstriana', 'MalvasiaNera', 'MalvasiadelLazio', 'MalvasiadiCandia', 'MalvasiadiLipari', 'MalvasiadiSchierano', 'MalvazijaIstarska', 'Mammolo', 'Mandilaria', 'Mandón', 'Manseng', 'Manteudo', 'MantoNegro', 'ManzoniBianco', 'Maratheftiko', 'MarechalFoch', 'MariaGomes', 'Marmajuelo', 'Marquette', 'Marsanne', 'Marselan', 'Marufo', 'Marzemino', 'Mataro', 'MaturanaBlanca', 'MaturanaTinta', 'MauzacBlanc', 'MauzacNoir', 'Mavro', 'MavroKalavritino', 'Mavrodafni', 'Mavrotragano', 'MavroudiArachovis', 'Mavrud', 'Mayolet', 'Mazuelo', 'Melnik', 'Melody', 'MelondeBourgogne', 'Mencia', 'Menoir', 'Merlot', 'Merseguera', 'Michet', 'Millot-Foch', 'MisketCherven', 'MisketVrachanski', 'ModrýPortugal', 'Molinara', 'Mollard', 'Monastrell', 'MondeuseNoire', 'Monica', 'Montepulciano', 'Montuni', 'Moradella', 'Morava', 'Morellino', 'Morenillo', 'Moreto', 'Morio-Muskat', 'Moristel', 'Moschofilero', 'Moschomavro', 'Mouhtaro', 'Mourisco', 'Mourvedre', 'MtsvaneKakhuri', 'Muscadelle', 'Muscadine', 'Muscardin', 'Muscat/MoscatelGalego', 'Muscat/MoscatelRoxo', 'Muscat/MoscateldeGranoMenudo', 'Muscat/MoscatelloSelvatico', 'Muscat/Moscato', 'Muscat/MoscatoBianco', 'Muscat/MoscatoGiallo', 'Muscat/MoscatoRosa', 'Muscat/MoscatodiScanzo', 'Muscat/Muscatel', 'Muscat/MuskatMoravsky', 'MuscatBaileyA', 'MuscatBlack', 'MuscatBlanc', 'MuscatEarly', 'MuscatGolden', 'MuscatNoir', 'MuscatOrange', 'MuscatOttonel', 'MuscatValvin', 'MuscatYellow', 'MuscatofAlexandria', 'MuscatofFrontignan', 'MuscatofHamburg', 'MuscatofSetúbal', 'MustoasadeMaderat', 'Müller-Thurgau', 'Narince', 'Nascetta', 'Nasco', 'Nebbiolo', 'Negoska', 'NegraraTrentino', 'NegraraVeronese', 'Negrette', 'Negroamaro', 'NegrudeDragasani', 'NerelloCappuccio', 'NerelloMascalese', 'NerettaCuneese', 'NeroBuonodiCori', 'NerodiTroia', 'Neuburger', 'Niagara', 'NiagaraBlanc', 'Nieddera', 'Nielluccio', 'Noble', 'Nocera', 'Noiret', 'Norton', 'Nosiola', 'Nouvelle', 'Nuragus', 'Ojaleshi', 'OlaszRizling', 'Ondenc', 'Orion', 'OrleansGelb', 'Ortega', 'Ortrugo', 'Oseleta', 'OtskhanuriSapere', 'Padeiro', 'Pagadebit', 'Palava', 'PallagrelloBianco', 'PallagrelloNero', 'Palomino', 'Pamid', 'Pampanuto', 'Parellada', 'Parraleta', 'Pascale', 'Passerina', 'Pavana', 'País/Mission', 'Pecorino', 'Pederna', 'Pedral', 'PedroXimenez', 'Pelaverga', 'Peloursin', 'Perera', 'Perle', 'Perricone', 'Perrum', 'PetitCourbu', 'PetitManseng', 'PetitMeslier', 'PetitRouge', 'PetitVerdot', 'PetiteArvine', 'PetiteMilo', 'PetitePearl', 'PetiteSirah', 'Peverella', 'Phoenix', 'Picardan', 'PiccolaNera', 'Picolit', 'PicpoulBlanc', 'Piedirosso', 'Pigato', 'Pignoletto', 'Pignolo', 'Pinenc', 'PinotAuxerrois', 'PinotBlanc', 'PinotGrigio', 'PinotGris', 'PinotMeunier', 'PinotNero', 'PinotNoir', 'Pinotage', 'PiquepoulBlanc', 'PiquepoulNoir', 'PlavacMali', 'PolleraNera', 'PosipBijeli', 'Poulsard', 'Premetta', 'Prensal', 'PretoMartinho', 'PrietoPicudo', 'Primitivo', 'Prié', 'Procanico', 'Prokupac', 'PrugnoloGentile', 'Pugnitello', 'Pulcinculo', 'Rabigato', 'RabodeOvelha', 'RabosoPiave', 'RabosoVeronese', 'Ramisco', 'Rebo', 'Refosco', 'RefoscodalPeduncoloRosso', 'Regent', 'Reichensteiner', 'RibollaGialla', 'Riesel', 'Rieslaner', 'Riesling', 'RieslingItálico', 'RieslingRenano', 'Ripolo', 'Rivaner', 'Rkatsiteli', 'Robola', 'Roditis', 'Roesler', 'Rolle/Rollo', 'Romeiko', 'Romé', 'Rondinella', 'Rondo', 'Roobernet', 'Roscetto', 'Rosetta', 'Rossese', 'Rossignola', 'Rossola', 'Rotberger', 'RoterVeltliner', 'Rotgipfler', 'Rougeon', 'Roupeiro', 'Roussanne', 'RoyaldeAlloza', 'Rubin', 'Rubired', 'Ruché', 'Ruen', 'Rufete', 'Ruggine', 'Ruländer', 'Räuschling', 'Sabrevois', 'Sacy', 'Sagrantino', 'Samsó', 'Sangiovese', 'Saperavi', 'Sarba', 'SauvignonBlanc', 'SauvignonGris', 'SavagninBlanc', 'Savatiano', 'Scheurebe', 'Schiava', 'SchiavaGentile', 'SchiavaGrigia', 'Schioppettino', 'Schwarzriesling', 'Schönburger', 'Sciacarello', 'Sciascinoso', 'SearaNova', 'Segalin', 'Seibel', 'Sercial', 'Sercialinho', 'SeyvalBlanc', 'ShirokaMelnishka', 'Sibirkovi', 'Sideritis', 'Siegerrebe', 'Silvaner/Sylvaner', 'Smederevka', 'Solaris', 'Sousão', 'SouvignierGris', 'Spätburgunder', 'St.Croix', 'St.Laurent', 'Steuben', 'Sultana', 'Sultaniye', 'Sumoll', 'SumollBlanc', 'Susumaniello', 'SwensonWhite', 'Symphony', 'Syrah/Shiraz', 'Syriki', 'Szürkebarát', 'Sémillon', 'Síria', 'TamaioasaRomaneasca', 'Tamarez', 'Tannat', 'Tarrango', 'Tazzelenghe', 'Tempranillo', 'TempranilloBlanco', 'Teroldego', 'Terrano', 'Terrantez', 'Terret', 'Thrapsathiri', 'Tibouren', 'Timorasso', 'TintaAmarela', 'TintaBarroca', 'TintaCaiada', 'TintaCarvalha', 'TintaFrancisca', 'TintaMadeira', 'TintaMiúda', 'TintaNegraMole', 'TintaRoriz', 'TintadeToro', 'TintadelPais', 'Tintilia', 'Tintilla', 'TintoCão', 'TintoFino', 'TintoreDiTramonti', 'TocaiFriulano', 'TocaiItalico', 'Torbato', 'Torrontés', 'TourigaFranca', 'TourigaNacional', 'Trajadura', 'Traminer', 'Traminette', 'Trebbiano', 'TrebbianoGiallo', 'TrebbianoRomagnolo', 'TrebbianoToscano', 'Treixadura', 'Trepat', 'Trincadeira', 'Triomphe', 'Trollinger', 'Trousseau', 'TsimlyanskyCherny', 'Tsolikouri', 'Turan', 'Turbiana', 'UghettadiCanneto', 'UgniBlanc', 'UlldeLlebre', 'UvaRara', 'Vaccareze', 'Valdiguie', 'ValentinoNero', 'Verdeca', 'Verdejo', 'Verdelho', 'Verdello', 'Verdicchio', 'Verdiso', 'VerduzzoFriulano', 'Vermentino', 'VermentinoNero', 'Vernaccia', 'VernacciadiOristano', 'VernacciadiSanGimignano', 'Vernatsch', 'Vespaiola', 'Vespolina', 'VidalBlanc', 'Vidiano', 'ViendeNus', 'Vignoles', 'Vijiriega', 'Vilana', 'VillardNoir', 'Vincent', 'Vinhão', 'Viognier', 'Violeta', 'Viorica', 'Viosinho', 'Vital', 'Vitovska', 'Viura', 'Vranac', 'Weissburgunder', 'Welschriesling', 'Xarel-lo', 'Xinomavro', 'Xynisteri', 'Zalema', 'Zelen', 'Zengö', 'Zibibbo', 'Zierfandler', 'Zinfandel', 'ZinfandelWhite', 'Zlahtina', 'Zweigelt', 'Zéta', 'ÁguaSanta', 'Öküzgözü']
    # Convert grapes list to set for faster lookup
    grapes_set = set(grapes)

    # Create a DataFrame with all grape columns populated with 0
    zeros_df = pd.DataFrame(0, index=X_pred.index, columns=grapes_columns)

    # Set 1 for the columns that are in the grapes list
    for grape in grapes:
        if grape in grapes_columns:
            zeros_df[grape] = 1

    # Concatenate X_pred with zeros_df along the columns axis
    X_pred = pd.concat([X_pred, zeros_df], axis=1)

    return X_pred


In [9]:
def pred(model, X_pred: pd.DataFrame):

    '''
    Return a dictonary that provides the predicted wines based on the provided X_pred
    '''

    y_pred = model.predict(X_pred)
    y_pred = y_pred.toarray()

    foods = ['Appetizer', 'Beef', 'Cured Meat', 'Game Meat', 'Lamb', 'Pasta', 'Pork', 'Poultry',
             'Rich Fish', 'Shellfish', 'Veal', 'Vegetarian']
    foods_index = np.where(y_pred[0]==1)[0].tolist()
    foods_to_choose = []
    for i in foods_index:
        foods_to_choose.append(foods[i])

    return {"foods": foods_to_choose}

# interface

## main.py

In [10]:
'''
!!! Uncomment next two lines in main.py !!!
'''
# from vinodine.ml_logic.data import create_binary_df
# from vinodine.ml_logic.model import create_X_train_y_train, train_model, save_model, open_model, create_X_pred, pred
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

In [28]:
file_path = "~/code/ArjanAngenent/VinoDine/raw_data/Cleaned_Full_100K_wines.csv"

model_save_path = 'models/model.pkl'

# Loading and binary encoding source data frame("grape_column": retrieving last column for binary encoded grapes to create X)
wine_df, grape_colum = create_binary_df(file_path)

X_train, X_test, y_train, y_test = create_X_train_y_train(wine_df, grape_column=grape_colum)

# Model training and saving
model = train_model(X_train, y_train)
save_model(model, model_save_path)

#if __name__ == '__main__': # remove "#" in python file and indent following lines
model = open_model(model_save_path)

y_pred = model.predict(X_test)

# Print classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Evaluate performance
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# create an new X_pred with manual input
X_pred = create_X_pred(Type='White',
                       Body='Full-bodied',
                       Acidity='High',
                       ABV='14.5',
                       grapes=['Colombard', 'Ugni Blanc'])

# predict foods for new input
y_pred_foods = pred(model, X_pred)

print(y_pred_foods)


Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.82      0.85      2602
           1       0.99      0.99      0.99     17390
           2       0.90      0.92      0.91      4952
           3       0.91      0.91      0.91     10912
           4       0.93      0.94      0.94     11641
           5       0.86      0.79      0.82      6743
           6       0.92      0.92      0.92      7656
           7       0.94      0.95      0.94     17504
           8       0.92      0.90      0.91      5575
           9       0.91      0.91      0.91      7469
          10       0.91      0.88      0.89      7469
          11       0.88      0.90      0.89      5491

   micro avg       0.93      0.92      0.92    105404
   macro avg       0.91      0.90      0.91    105404
weighted avg       0.93      0.92      0.92    105404
 samples avg       0.93      0.92      0.92    105404

Accuracy: 0.8160045746577416
{'foods': ['Appetizer', 'Pa

# API

## fast.py

In [32]:
'''
!!! Uncomment next two lines !!!
'''
# from vinodine.ml_logic.model import open_model, create_X_pred, pred
from fastapi import FastAPI, Query
from fastapi.middleware.cors import CORSMiddleware
from typing import List


In [33]:
app = FastAPI()
app.add_middleware(CORSMiddleware,
                   allow_origins=["*"],
                   allow_methods=["*"],
                   allow_headers=["*"])


model_open_path = 'model.pkl'
app.state.model = open_model(model_open_path)


@app.get('/')
def root():
    return {'greeting': 'Hello'}


@app.get('/predict')
def predict(Type: str,
            ABV: float,
            Body: str,
            Acidity: str,
            grapes: List[str] = Query('grape')): # allows to pass a list of strings(grapes) as params in POST request
    # create managebale X_pred for model from API request
    X_pred = create_X_pred(Type = Type,
                           ABV = ABV,
                           Body = Body,
                           Acidity = Acidity,
                           grapes = grapes)

    # return suggested foods based on pretrained model and X_pred created from API request
    y_pred_foods = pred(app.state.model, X_pred)

    return y_pred_foods